# Ollama

**Running LLM's locally - Master Applied AI - Michiel Bontenbal - 12 december 2024**

Ollama is a tool that allows users to run open-source large language models (LLMs) locally on your laptop. Ollama supports a variety of models, including Llama2, Mistral, CodeLlama and many others. 

You'll need to download ollama first. Download it from www.ollama.com.

Courtesy of some code examples to ollama.com / Jeffrey Morgan.
License: MIT License

### Contents
0. Install and settings
1. First script
2. Streaming the response
3. Create a gradio front end

### Sources
- https://github.com/ollama/ollama-python
- https://github.com/ollama/ollama/blob/main/docs/api.md#api
- https://pypi.org/project/ollama/


## 0. Install and settings

*Before running this code, make sure you've installed ollama on your laptop!*

In [ ]:
# Check your version of python. To run ollama with python you will need Python 3.8 or higher.
from platform import python_version
print(python_version())

In [ ]:
#before downloading the model check available disk space. You will need at least 20 Gb!
import shutil
usage = shutil.disk_usage("/")
free_space_bytes = usage.free
free_space_gb = free_space_bytes / (1024 * 1024 * 1024)  # Convert to GB
print(f'free disk space = {round(free_space_gb,1)} Gb')

In [ ]:
#Check processor and RAM
import psutil
import platform
print("Processor:", platform.processor())
memory = psutil.virtual_memory()
print(f'Total RAM: "{memory.total/1000000000} Gb')
print(f"Available RAM: {memory.available/1000000000} Gb")
print(f"RAM Usage: {memory.percent}%")

In [ ]:
%pip install --upgrade ollama

In [ ]:
# Make sure you run from harddisk. Running this from OneDrive or cloud makes it much slower.
import os
print(f"Current working directory: {os.getcwd()}")

In [ ]:
#download a model from the ollama server. May take a minute... Uncomment if necessary
import ollama
ollama.pull('llama3.2:1b')

In [ ]:
#get all the models on your device
ollama.list()

In [ ]:
#Let's unpack it a bit (ollama changed it's API this week...) so 
models = ollama.list()
print(models)
modellen = models.models
for i in range (len(modellen)):
    print(models.models[i].model)

In [ ]:
#printing the details of a model
ollama.show('llama3.2:1b')

In [ ]:
#show all functions
print(dir(ollama))

In [ ]:
#Delete a model. 
#ollama.delete(<your model>) #replace <your model>

## 1. Run first script

In [ ]:
#first set the model
model = 'llama3.2:1b'

In [ ]:
#first script from ollama website (https://github.com/ollama/ollama-python)
import ollama
response = ollama.chat(model=model, messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [ ]:
#Create the ollama function
import ollama

def ask_ollama(question, model):
    """
    
    Sends a question to the Ollama API and returns the response.
    """
    response = ollama.chat(
        model=model,
        messages=[
            {'role': 'user', 'content': question},
        ],
    )

    return response['message']['content']

# Example usage
response_content = ask_ollama("Why is the sky blue?", model)
print(response_content)

## 2. Streaming the response

With streaming the response will be printed on the screen while the LLM is still busy generating the answer. This is a faster solution. Try it out yourself!

In [ ]:
question = input('Your question:')

In [ ]:
#same but now as a function (to use with gradio) 
import ollama

def ollama_chat_stream(question):
    """
    Streams the chat response from Ollama using the 'tinyllama' model.
    """
    # Initialize the chat with Ollama
    stream = ollama.chat(
        model=model,
        messages=[{'role': 'user', 'content': question}],
        stream=True,
    )

    # Stream and print the responses
    for chunk in stream:
        print(chunk['message']['content'], end='', flush=True)
        #print(chunk['message']['content'], end='', flush=True)

# Example usage
ollama_chat_stream(question)


## 3. Creating a gradio front end

Gradio is a very high level Python library that let's you create a front-end very quickly. It is used to demo your model. Gradio starts a server for you (like Flask or NodeJS).

In [ ]:
#uncomment if necessary
!pip install gradio --upgrade

In [ ]:
import gradio

In [ ]:
#a Gradio frontend make sure you have run previous cells
import gradio as gr

iface = gr.Interface(
    fn=ask_ollama,  #use the function we defined under 1
    inputs="text", 
    outputs= "text"
)

iface.launch()